In [1]:
import numpy as np

Source is from here: http://mlwiki.org/index.php/Gram-Schmidt_Process#QR_Factorization Approach is similar to the one used in gram-schmidt orthogonization.

In [2]:
def qr_factorization(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j]

        for i in range(j - 1):
            q = Q[:, i]
            R[i, j] = q @ v
            v = v - R[i, j] * q

        norm = np.linalg.norm(v)
        Q[:, j] = v / norm
        R[j, j] = norm
    return Q, R

Example usage

In [3]:
np.random.seed(1)
A = np.random.rand(13, 10) * 1000
Q, R = qr_factorization(A)
Q.shape, R.shape

((13, 10), (10, 10))

In [4]:
Q

array([[ 0.21576639,  0.32534481, -0.13474949, -0.1982346 , -0.11620693,
        -0.22758271, -0.01727056,  0.2870567 ,  0.18688782,  0.21318668],
       [ 0.21689045,  0.30948914, -0.01652058,  0.33620784, -0.25417341,
         0.17596279, -0.27914355,  0.04370379, -0.12986738, -0.1753672 ],
       [ 0.41430372,  0.43732913, -0.07644399, -0.1971675 ,  0.20657614,
         0.22795572, -0.45210167, -0.78427934, -0.20836739, -0.12401619],
       [ 0.05088447,  0.19019925,  0.52572879,  0.3290194 ,  0.15619418,
        -0.4036995 , -0.14944031, -0.10087158, -0.42215658, -0.25647463],
       [ 0.51163485,  0.33791967, -0.15645436, -0.21683432, -0.36218165,
        -0.30264908,  0.26022222, -0.02097441, -0.31506599, -0.67584227],
       [ 0.01002043,  0.30660573,  0.11691339,  0.09650501,  0.1820355 ,
        -0.23425816,  0.20300451, -0.22624081,  0.32624655,  0.17952661],
       [ 0.05294764,  0.18701428,  0.37108034,  0.21778351, -0.24801254,
         0.01417317,  0.02014723, -0.17467187

In [5]:
R

array([[1932.74770646,    0.        , 1073.5214862 , 1694.29308965,
         998.04062526, 1489.16011078, 1465.02945759, 1318.94969391,
        1226.31343579, 1508.42701128],
       [   0.        , 2214.03405123,    0.        ,  468.35978253,
         530.64054053,  415.8946801 ,  498.57154768,  549.27252153,
         238.27453968,  743.80131529],
       [   0.        ,    0.        , 1718.11775471,    0.        ,
         626.58458006,  857.42896996,  783.70518891, 1405.32853401,
         811.95076732,  841.6251716 ],
       [   0.        ,    0.        ,    0.        , 1087.68784575,
           0.        ,  221.98725953,  834.51541984,  342.15625888,
          87.88034348,  141.04070641],
       [   0.        ,    0.        ,    0.        ,    0.        ,
        1349.29073083,    0.        ,   10.67641691,  381.76061915,
         -18.7529284 ,  865.95061231],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,  899.61853644,    0.        ,  379

In [6]:
np.abs((A - Q @ R).sum()) < 1e-6

True

Second version using Householder Transformation

In [7]:
def qr_householder(A):
    m, n = A.shape
    Q = np.eye(m) # Orthogonal transform so far
    R = A.copy() # Transformed matrix so far

    for j in range(n):
        # Find H = I - beta*u*u' to put zeros below R[j,j]
        x = R[j:, j]
        normx = np.linalg.norm(x)
        rho = -np.sign(x[0])
        u1 = x[0] - rho * normx
        u = x / u1
        u[0] = 1
        beta = -rho * u1 / normx

        R[j:, :] = R[j:, :] - beta * np.outer(u, u) @ R[j:, :]
        Q[:, j:] = Q[:, j:] - beta * Q[:, j:] @ np.outer(u, u)
        
    return Q, R

In [8]:
Q_h, R_h = qr_householder(A)
Q_h.shape, R_h.shape

((13, 13), (13, 10))

In [9]:
np.abs((A - Q_h @ R_h).sum()) < 1e-6

True

In [10]:
np.sqrt((12*12)+(6*6)+(4*4))

14.0

In [11]:
((2*2)+(6*6)+(4*4))

56

In [12]:
(1)+(3*3)+(2*2)

14